# Pdf exploration and preparation test

Main goal is to check if we can read the pdf, extract only relevant content, see how we can post-process the extracted text and finally have some informations about the extracted text.

Links to `Simple Local RAG Tutorial` :
* [GitHub](https://github.com/mrdbourke/simple-local-rag) ;
* [YouTube](https://youtu.be/qN_2fnOPY-M?si=APnkpsGY0z_scJ9Z).

In [3]:
import re
from pathlib import Path
from pprint import pprint

import pandas as pd
import pdfplumber
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

## Extract the pdf pages

### Set the file path

In [7]:
PDF_FILENAME = "source.pdf"

p = Path()
p = p.resolve() / "pdf"
q = p / PDF_FILENAME

if q.is_file():
    print(f"Pdf file path : '{q}'.")
else:
    print("No pdf file found.")

Pdf file path : '/home/thomas/gcp-professional-data-engineer-rag/pdf/source.pdf'.


### Read the pdf

Let's see if the number of pages found is the right one.

In [4]:
with pdfplumber.open(q) as pdf:
    print(
        f"* Expected number of pages : \t355\n* Number of pages found : \t{len(pdf.pages)}"
    )

* Expected number of pages : 	355
* Number of pages found : 	355


That's ok, we can try to extract text from a random test page.

In [5]:
with pdfplumber.open(q) as pdf:
    page = pdf.pages[101]
    text = page.extract_text()
    print(text[:90])

Data pipelines are sequences of operations that copy, trans-
form, load, and analyze data.


The extraction works but the text doesn't correspond to the one in the selected page above. First thing to take in account is the fact that the first item in a list is at index 0. So when we write `page = pdf.pages[101]`, in fact it is the page 102 which is extracted.
But it is still not ok : the extracted text correpond to page 62 which means page 1 in the pdf is actually the page 41 (index 40). The reason is all the "About", "Introduction", etc. sections are not numbered the same way in the pdf file.
This is something to take in account to extract the desired content.

### Target relevant text

Documents can have several information which are not relevant to build a RAG :
* headers and footers ;
* tables ;
* hyperlink ;
* figures ;
* etc..

We only want to keep the body of the document but also the code samples even if a part of this last is not always relevant. Since each document is different, there is not a unique method to determine what is relevant or not. The only way to handle this is to take time to inspect the document structure, layout, etc..

In my case, it appears that the **font** will be the best way to help me target the body and the code.

> Take note that working with fonts means we will extract the text character by character to access its properties thanks to the [`chars` object](https://github.com/jsvine/pdfplumber?tab=readme-ov-file#objects) available for each instance of `pdfplumber.PDF` and `pdfplumber.Page`.

In [6]:
with pdfplumber.open(q) as pdf:
    page = pdf.pages[43]
    header_font = page.chars[3].get("fontname")
    body_font = page.chars[103].get("fontname")
    print(f"* Header fontname : \t{header_font}\n* Body fontname : \t{body_font}")

* Header fontname : 	GHSRZR+UniversLTStd
* Body fontname : 	GHSRZR+SabonLTStd-Roman


In [7]:
with pdfplumber.open(q) as pdf:
    page = pdf.pages[50]
    code_font = page.extract_text_lines(return_chars=True)[8]["chars"][0].get(
        "fontname"
    )
    print(f"* Sample code fontname : \t{code_font}")

* Sample code fontname : 	GHSRZR+SourceCodePro-Regular


Header, body and code have different fonts which is of great help. The last thing to take care of is the fact that the text we want to target can be *italic* or **bold**. So let's make a list of all available fonts in the file.

In [8]:
# Extract all fonts in the document
fontname_list = []
with pdfplumber.open(q) as pdf:
    for page in pdf.pages:
        [
            fontname_list.append(char.get("fontname"))
            for char in page.chars
            if char.get("fontname") not in fontname_list
        ]

In [11]:
# List only the necessary fonts
body_fontname_list = [
    fontname
    for fontname in fontname_list
    if "Sabon" in fontname or "SourceCode" in fontname
]
print(body_fontname_list)

['GHSRZR+SabonLTStd-Roman', 'GHSRZR+SourceCodePro-Regular', 'GHSRZR+SabonLTStd-Bold', 'GHSRZR+SabonLTStd-Italic', 'URTXBU+SourceCodePro-Bold']


Last step for the font part : we will create a helper function to filter the extracted text by font using the fontname of each character.

In [12]:
# Font filter helper funtion
def filter_text_by_font(chars: list[dict], target_fonts: list[str]) -> str:
    """Filters extracted text and, more precisely, its letters by their fonts.

    Args:
        chars (list[dict]): chars object from pdfplumber.
        target_fonts (list[str]): list of fontnames for which we want to keep the characters/text.

    Returns:
        str: filtered text.
    """
    char_text = [char["text"] for char in chars if char.get("fontname") in target_fonts]
    text = "".join(char_text)
    return text

### Text post-processing

#### Basic formatting

The goal is to have the cleanest text as possible for further steps. We will remove uppercase and unecessary spaces. In addition to that, we will also replace *fifi* string by *fi*. This is a specific error I noticed after the extraction of my document which shows how important it is to inspect each document carefully to identify the best way to process it.
Here is a sample text.

In [23]:
# Basic text formatter function
def basic_text_formatter(text: str) -> str:
    """Applies different operations to format and clean the text.

    Args:
        text (str): original text.

    Returns:
        str: formatted text.
    """
    formatted_text = " ".join(
        text.casefold().replace("\n", " ").replace("fifi", "fi").split()
    )
    return formatted_text

In [26]:
basic_text_sample = " I'm a Basic   text sample. "

print(
    f"* Before : \t{basic_text_sample}\n* After : \t{basic_text_formatter(basic_text_sample)}"
)

* Before : 	 I'm a Basic   text sample. 
* After : 	i'm a basic text sample.


#### Hyphens

Hyphens are used to break words so that the appearance of the page is nicer but it will interfere in the words recognition.

In [27]:
with pdfplumber.open(q) as pdf:
    page = pdf.pages[237]
    text = page.extract_text()
    hyphen_text_sample = text[1066:1078]
    print(hyphen_text_sample)

con-
necting


In [17]:
def remove_hyphens(text: str) -> str:
    """Removes hyphens from text.

    Args:
        text (str): original text.

    Returns:
        str: processed text.
    """
    lines = [line.rstrip() for line in text.split("\n")]

    # Find dashes
    line_numbers = []
    for line_no, line in enumerate(lines[:-1]):
        if line.endswith("-"):
            line_numbers.append(line_no)

    # Replace
    for line_no in line_numbers:
        lines = dehyphenate(lines, line_no)

    return " ".join(lines)


def dehyphenate(lines: list[str], line_no: int) -> list[str]:
    """Rebuilds lines (words) separated by hyphen.

    Args:
        lines (list[str]): lines to process.
        line_no (int): index of lines to process.

    Returns:
        list[str]: list of modified lines.
    """
    next_line = lines[line_no + 1]
    word_suffix = next_line.split(" ")[0]

    lines[line_no] = lines[line_no][:-1] + word_suffix
    lines[line_no + 1] = lines[line_no + 1][len(word_suffix) :]
    return lines

In [18]:
print(
    f"* Before : \t{hyphen_text_sample}\n* After : \t{remove_hyphens(hyphen_text_sample)}"
)

* Before : 	con-
necting
* After : 	connecting 


### Text extraction

We now have all our "tools"" to extract the pdf pages correctly and in a relevant way. To refine a bit more our target will remove the pages we don't want to keep (like introduction, glossary, etc.) and we will skip the blank pages (with no content).

Do do this, We will write a final function to process our whole document. Pages will be stored in a list of dictionnaries where we will be able to add information like page number, number of characters, tokens, sentences, etc.. and to explore the pages information by converting it to a DataFrame.

In [126]:
def extract_and_process_pdf(path: Path) -> list[dict]:
    """Open a pdf file with pdfplumber, extracts and formats relevant pages then append
    their content and statistics in a list.

    Args:
        path (Path): Pathlib path of the document.

    Returns:
        list[dict]: Extracted content and informations of pages.
    """
    extracted_pages = []

    with pdfplumber.open(path) as pdf:
        for page_idx, page in enumerate(pdf.pages):
            page_number = page_idx - 39
            lines = page.extract_text_lines(return_chars=True, keep_blank_chars=True)

            kept_lines = []
            for line in lines:
                kept_lines.append(
                    filter_text_by_font(line["chars"], body_fontname_list)
                )
            text = "\n".join(kept_lines)

            text = remove_hyphens(text)
            text = basic_text_formatter(text)

            if 0 < page_number <= 305 and text:
                extracted_pages.append(
                    {
                        "page_number": page_number,
                        "page_chars_count": len(text),
                        "page_words_count": len(text.split(" ")),
                        "page_raw_sentences_count": len(re.split(r"[.?!]", text)),
                        "page_text": text,
                    }
                )

    return extracted_pages

In [168]:
extracted_pages = extract_and_process_pdf(q)

## Splitting pages text

### Number of tokens

First of all we must take in account the fact that we will use the `sentence-transformers` model [`all-mpnet-base-v2`](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) which as *a max tokens capacity of **514***. The token capacity *is very a important concept for a model* since it refers to *the maximum number of tokens it can process* in its context window during a single interaction.

In our case, the `all-mpnet-base-v2` model has been trained to ingest and turn into embeddings texts with 514 tokens. Texts over 514 tokens which are encoded by this model will be automatically reduced to 514 tokens in length, potentially losing some information.

So what we want to know is how many tokens we have per page. We wil start by a raw tokens counts using the method explained [here](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) or [here](https://python.langchain.com/docs/concepts/tokens/).

In [170]:
# Convert the extracted pages dictionnaries to DataFrame
df = pd.DataFrame(extracted_pages)

# Generate descriptive statistics
df.describe().drop(columns=["page_number"]).loc[["mean", "min", "max"]].round(2)

,page_chars_count,page_words_count,page_raw_sentences_count
mean,2082.93,331.97,21.32
min,121.00,22.00,1.00
max,3705.00,624.00,55.00


We can know add the `raw_token_count`.

In [171]:
# Calculate raw tokens count
for page in extracted_pages:
    page["page_raw_tokens_count"] = len(page["page_text"]) // 4

# Regenerate DataFrame and statistics
df = pd.DataFrame(extracted_pages)
df.describe().drop(columns=["page_number"]).loc[
    ["mean", "50%", "75%", "min", "max"]
].round(2)

,page_chars_count,page_words_count,page_raw_sentences_count,page_raw_tokens_count
mean,2082.93,331.97,21.32,520.38
50%,2161.00,341.00,20.00,540.00
75%,2615.00,414.00,26.00,653.00
min,121.00,22.00,1.00,30.00
max,3705.00,624.00,55.00,926.00


Looking at the results we can see that the average raw count per page is above the tokens capacity of our model. But is only a raw count so let's try to be more precise.

For the next steps to come, we will use the *LangChain* framework and its different tools. We will instantiate a `SentenceTransformersTokenTextSplitter` and use the `count_token` method. What is nice with this solution is the fact that its a specialized text splitter for use with `sentence-transformer` models. This means it will behave taking in account the model we will use.

In [ ]:
# Instantiate the text splitter
text_splitter = SentenceTransformersTokenTextSplitter(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# Calculate real tokens count
for page in extracted_pages:
    page["page_real_tokens_count"] = text_splitter.count_tokens(text=page["page_text"])

# Regenerate DataFrame and statistics
df = pd.DataFrame(extracted_pages)
df.describe().drop(columns=["page_number"]).loc[
    ["mean", "50%", "75%", "min", "max"]
].round(2)

,page_chars_count,page_words_count,page_raw_sentences_count,page_raw_tokens_count,page_real_tokens_count
mean,2082.93,331.97,21.32,520.38,431.07
50%,2161.00,341.00,20.00,540.00,436.00
75%,2615.00,414.00,26.00,653.00,544.00
min,121.00,22.00,1.00,30.00,29.00
max,3705.00,624.00,55.00,926.00,785.00


This is better. The average tokens count is below the capacity of the model but *we still have 25 % of pages with more than 544 tokens*.

### Creating chunks

The next step will be to split our text in manageable chunks with the right amount of token. And will to that with using the `split_text` method of `SentenceTransformersTokenTextSplitter`.

When splitting the text, we will configure a chunk overlap which define the number of characters which overlap between chunks ensuring that context is preserved. Take in mind that increasing the overlap will increase the number of chunks created.

In [186]:
# Instantiate the text splitter
text_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=200, model_name="sentence-transformers/all-mpnet-base-v2"
)

# Split the text for each page
for page in extracted_pages:
    page["page_chunks"] = text_splitter.split_text(text=page["page_text"])
    page["page_chunks_max_tokens_count"] = max(
        text_splitter.count_tokens(text=chunk) for chunk in page["page_chunks"]
    )
    page["page_chunks_count"] = len(page["page_chunks"])

# Regenerate DataFrame and statistics
df = pd.DataFrame(extracted_pages)
df.describe().drop(columns=["page_number"]).loc[
    ["mean", "50%", "75%", "min", "max"]
].round(2)

,page_chars_count,page_words_count,page_raw_sentences_count,page_raw_tokens_count,page_real_tokens_count,page_chunks_max_tokens_count,page_chunks_count
mean,2082.93,331.97,21.32,520.38,431.07,343.94,1.82
50%,2161.00,341.00,20.00,540.00,436.00,386.00,2.00
75%,2615.00,414.00,26.00,653.00,544.00,386.00,2.00
min,121.00,22.00,1.00,30.00,29.00,29.00,1.00
max,3705.00,624.00,55.00,926.00,785.00,389.00,4.00


Nice the number of tokens now fits with the model capacity, we are ready to embed.

### Embeddings chunks

#### Test

In [241]:
# Instantiate the embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": None},
    encode_kwargs={"normalize_embeddings": True},
)

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer.",
]

# Embed sentences
embeddings = embeddings.embed_documents(sentences)

In [246]:
for embedding in embeddings:
    print("len_embedding : ", len(embedding))
    print("embedding : ", embedding)

len_embedding :  768
embedding :  [-0.02079826593399048, 0.03031645342707634, -0.020121803507208824, 0.06864846497774124, -0.025525597855448723, -0.00847684871405363, -0.0002072092320304364, -0.06323777884244919, 0.02816072665154934, -0.033335354179143906, 0.030263422057032585, 0.05307215079665184, -0.05035267397761345, 0.02622886933386326, 0.03333140164613724, -0.04515775665640831, 0.03630451112985611, -0.0013711987994611263, -0.012017090804874897, 0.01149469893425703, 0.050451066344976425, 0.047085702419281006, 0.02119137905538082, 0.0514606349170208, -0.020374663174152374, -0.03588895499706268, -0.0006677887286059558, -0.029439402744174004, 0.04958593472838402, -0.010563965886831284, -0.015201432630419731, -0.0013176074717193842, 0.04481975734233856, 0.015602296218276024, 8.603792025496659e-07, -0.001213915296830237, -0.023797865957021713, -0.0009093762491829693, 0.007344856858253479, -0.002539339940994978, 0.05233703553676605, -0.04680434986948967, 0.016621500253677368, 0.047157965